Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist ein Service **Webshop** am laufen, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

- - -

Zuerst erstellen wir den Kubernetes Namespace ms-rest und Labeln diesen, dass Kiali Informationen sammeln kann.

In [ ]:
! kubectl create namespace ms-rest
! kubectl label  namespace ms-rest istio-injection=enabled

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort im Pulldownmenu den Namespace "ms-rest" auszuwählen.

Wählt nachfolgenden Link an und aktzeptiert das Zertifikat um dann ohne Token, drückt "Überspringen" oder "Skip", ins Dashboard zu wechseln.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"

Der Webshop, aus den vorherigen Übungen, ist jetzt aufgeteilt in einzelne Microservices.

Diese starten wir deklarativ mittels `kubectl apply -f <Deklaration>`

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/catalog-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/customer-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/order-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml
kubectl get pod,services --namespace ms-rest

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

### Zipkin

Zipkin ist ein verteiltes tracing system. Es hilft beim Sammeln von Zeitdaten, die zur Behebung von Latenzproblemen in Service-Mesh erforderlich sind.  

In [ ]:
%%bash
kubectl get service -n istio-system -l name=zipkin -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
echo "Zipkin  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/zipkin -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

In [ ]:
%%bash
URL="http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

for i in `seq 1 1000`;
do
    curl -s -o /dev/null -w "%{http_code}" ${URL}/order/order
    echo
done

- - - 
### REST Paradigma

Representational State Transfer (abgekürzt REST, seltener auch ReST) bezeichnet ein Programmierparadigma für verteilte Systeme.

REST ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web (HTTP GET, PUT, POST, DELETE).

REST hat das Ziel, einen Architekturstil zu schaffen, der die Anforderungen des modernen Web besser darstellt. Dabei unterscheidet sich REST vor allem in der Forderung nach einer einheitlichen Kommunikation Schnittstelle von anderen Architekturstilen.

Um das REST Paradigma zu verdeutlichen, führen wir mittels `curl` (Windows `Invoke-WebRequest`) mehrere API Aufrufe gegen unsere Microservices aus:

In [ ]:
%%bash
URL="http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop
curl -s ${URL}/customer/customer/api | jq

In [ ]:
%%bash
URL="http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop
curl -s ${URL}/catalog/catalog/api | jq

In [ ]:
%%bash
URL="http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop
curl -s ${URL}/order/order/api | jq

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-rest

- - -
### Quellen

* Sourcecode: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v2.1.0?ref_type=heads
* Kubernetes Deklarationen: https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates
* Container Registry: https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry